In [2]:
from openai import OpenAI

In [3]:
openai_client = OpenAI()

In [4]:
response = openai_client.responses.create(
    model="gpt-4o-mini",
    input="Write a short bedtime story about a unicorn"
)

print(response.output_text)

**The Starry Night Unicorn**

Once upon a time, in a magical forest filled with sparkling flowers and whispering trees, there lived a gentle unicorn named Luna. Her coat shimmered like the moonlight, and her horn glowed softly, illuminating the paths she trotted.

Every evening, as the sun dipped beyond the horizon, Luna would watch the stars twinkle into existence. She believed that each star held a special wish, waiting for someone to believe. But Luna had a secret: she had never made a wish of her own.

One night, when the sky was particularly bright, Luna spotted a little girl named Mia, who had climbed a tall hill to see the stars. Mia looked sad, and Luna trotted over, her horn glowing softly. 

“Hello, little one,” Luna said gently. “Why do you look so sad?”

Mia sighed. “I wished for a friend to share the stars with, but I’m all alone.”

Luna’s heart felt warm. She lowered her head and said, “Close your eyes and make a wish.”

Mia closed her eyes tightly and whispered her wish 

We typically use messages for that instead of putting everything in just one string:

In [5]:
messages = [
    {"role": "user", "content": "tell me a joke about Alexey"}
]

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=messages
)

In [6]:
print(response.model_dump_json(indent=2))

{
  "id": "resp_07e68293d482dc7f0068e73fdfde20819b938ce32f7b90e525",
  "created_at": 1759985631.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-4o-mini-2024-07-18",
  "object": "response",
  "output": [
    {
      "id": "msg_07e68293d482dc7f0068e73fe14418819b91eab71a77e96641",
      "content": [
        {
          "annotations": [],
          "text": "Why did Alexey bring a ladder to the bar?\n\nBecause he heard the drinks were on the house!",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [],
  "top_p": 1.0,
  "background": false,
  "conversation": null,
  "max_output_tokens": null,
  "max_tool_calls": null,
  "previous_response_id": null,
  "prompt": null,
  "prompt_cache_key": null,
  "reasoning": {
    "effort": nu

What is interesting here:

* output - everything that the LLM returned

* usage - how many tokens we used (also - how much we paid)

Getting the text:

* response.output[0].content[0].text

* response.output_text

We can stream the response:

In [10]:
stream = openai_client.responses.create(
    model='gpt-4o-mini',
    input=messages,
    stream=True
)

for event in stream:
    print(event)

ResponseCreatedEvent(response=Response(id='resp_0f253b3c9d667a710068e74082470c819b84e150799b5f7988', created_at=1759985794.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tier='auto', status='in_progress', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), top_logprobs=0, truncation='disabled', usage=None, user=None, store=True), sequence_number=0, type='response.created')
ResponseInProgressEvent(response=Response(id='resp_0f253b3c9d667a710068e74082470c819b84e150799b5f7988', created_at=1759985794.0, error=None, incomplete_details=None, instru

In [11]:
for event in stream:
    if hasattr(event, 'delta'):
        print(event.delta, end='')

In [12]:
system_prompt = """
You're an assistant that can make jokes. Always find out the name of
the person to make the jokes personalized. Once you know the name,
make the joke about them.
""".strip()

messages = [
    {"role": "developer", "content": system_prompt},
    {"role": "user", "content": "tell me a joke"}
]

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=messages
)

print(response.output_text)

Sure! But first, what's your name?


In [15]:
messages.append(response.output[0])
messages.append({"role": "user", "content": "Iñigo"})

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=messages
)

print(response.output_text)

Nice to meet you, Iñigo! Here’s one for you:

Why did Iñigo bring a pencil to the party?

Because he wanted to draw some attention!


In [16]:
from toyaikit.llm import OpenAIClient
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIResponsesRunner

system_prompt = """
you're an assistant that can make jokes. Always find out the name of
the person to make the jokes personalized. Once you know the name,
make the joke about them.
""".strip()

llm_client = OpenAIClient(
    model='gpt-4o-mini',
    client=openai_client
)

runner = OpenAIResponsesRunner(
    tools=None,
    developer_prompt=system_prompt,
    chat_interface=IPythonChatInterface(),
    llm_client=llm_client
)

runner.run();

You: Iñigo


You: stop


Chat ended.
